In [32]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.axes as axes
import copy
import random
import time
from random import uniform, choice
import numpy as np
from numpy import dot
from sklearn import datasets, svm, metrics
from sknn.mlp import Classifier, Convolution, Layer

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [19]:
class CellularAutomaton:
    
    Colors = {0:"gray", 1:"cyan"}
    
    def __init__(self, dimens):
        self.grid = [[0 for x in range(dimens[1])] for y in range(dimens[0])]
        self.width = dimens[0]
        self.height = dimens[1]
        
    def __eq__(self, other):
        return self.grid == other.grid
    
    def __neq__(self, other):
        return not self == other
    
    def __hash__(self):
        return hash(str(self.grid))%100000
    
    def __cmp__(self, other):
        if self == other:
            return 0
        return __cmp__(id(self), id(other))
    
    def __lt__(self, other):
        return False
    
    def clone(self):
        return copy.deepcopy(self)
    
    def draw(self, compare):
        fig1 = plt.figure()
        ax1 = fig1.add_subplot(1,2,1, aspect='equal')
        ax2 = fig1.add_subplot(1,2,2, aspect='equal')
        ax1.set_axis_bgcolor('gray')
        ax1.set_xlim([0,self.width])
        ax1.set_ylim([0,self.height])
        ax2.set_axis_bgcolor('gray')
        ax2.set_xlim([0,len(compare)])
        ax2.set_ylim([0,len(compare[0])])
        for x in range(0, self.width):
            for y in range(0, self.height):
                cell = self.grid[x][y]
                if cell == 0: continue
                ax1.add_patch(
                    patches.Rectangle((x, y),
                                      1,1,
                                      fill=True,
                                      facecolor=CellularAutomaton.Colors[cell],
                                      edgecolor="black",
                                      hatch= None,
                                      label=cell)
                    )
        for x in range(0, len(compare)):
            for y in range(0, len(compare[0])):
                cell = compare[x][y]
                if cell == 0: continue
                ax2.add_patch(
                    patches.Rectangle((x, y),
                                      1,1,
                                      fill=True,
                                      facecolor=CellularAutomaton.Colors[cell],
                                      edgecolor="black",
                                      hatch= None,
                                      label=cell)
                    )
        plt.show(fig1)
                

#example = CellularAutomaton((30,30))
#example.grid[2][2] = 1
#example.draw()

In [3]:
class GameOfLife(CellularAutomaton):
    
    def __init__(self, dimens):
        CellularAutomaton.__init__(self, dimens)
        self.turns = 0
            
    def num_neighbors(self, pos):
        neighbors = 0
        for i in range(-1,2):
            for j in range(-1,2):
                x = pos[0] + i
                y = pos[1] + j
                if (i is not 0 or j is not 0) and x >= 0 and y >= 0 and x < self.width and y < self.height:
                    if self.grid[x][y] is not 0:
                        neighbors += 1
        return neighbors
    
    def become_alive(self, pos):
        neighbors = self.num_neighbors(pos)
        if neighbors == 3:
            return True
        else:
            return False
    
    def check_alive(self, pos):
        neighbors = self.num_neighbors(pos)
        if neighbors < 2 or neighbors > 3:
            return False
        else:
            return True
    
    # 1 2 3
    # 4 5 6
    # 7 8 9
    def get_training_data(self, pos):
        neighbors = [[0 for y in range(0,3)] for x in range(0,3)]
        count1 = 0
        count2 = 0
        for j in range(1,-2, -1):
            for i in range(-1,2):
                x = pos[0] + i
                y = pos[1] + j
                if x < 0 or y < 0 or x >= self.width or y >= self.height or self.grid[x][y] is 0:
                    neighbors[count1][count2] = 0
                else:
                    neighbors[count1][count2] = self.grid[x][y] 
                count2 += 1
            count2 = 0
            count1 += 1
        return neighbors
    
    def play(self):
        outputs = np.array([0 for x in range(self.width*self.height)])
        new_game = self.clone()
        count = 0
        for x in range(0, self.width):
            for y in range(0, self.height):
                cell = self.grid[x][y]
                if cell == 0:
                    if self.become_alive((x,y)):
                        new_game.grid[x][y] = 1
                elif cell == 1:
                    if not self.check_alive((x,y)):
                        new_game.grid[x][y] = 0
                outputs[count] = new_game.grid[x][y]
                count += 1
        
        self.grid = new_game.grid
        self.turns += 1
        return outputs
        
    def get_inputs(self):
        inputs = np.array([[[0, 0, 0] for x in range(0,3)] for i in range(self.width*self.height)])
        count = 0
        for x in range(0, self.width):
            for y in range(0, self.height):
                inputs[count] = self.get_training_data((x,y))
                count += 1
        return inputs

Example Game Of Life Graph

In [4]:
def make_game():
    game = GameOfLife((10, 10))
    for x in range(0, game.width):
        for y in range(0, game.height):
            num = random.randint(0,2)
            if num == 0:
                game.grid[x][y] = 1
            else:
                game.grid[x][y] = 0
    return game

compare = [[0 for x in range(0,20)] for y in range(0,20)]
#game.draw(compare)


#for x in range(0, 1):
#    game.play()
#    game.draw(compare)
#game = GameOfLife((20,20))
#game.grid[0][0] = 1
#game.grid[1][0] = 1
#game.grid[1][1] = 1
#game.draw(compare)

In [5]:
game = make_game()
training = game.get_inputs()
print(training)
#game.draw(compare)
outputs = game.play()
print(outputs)
#game.draw(compare)

[[[0 1 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 1]]

 [[0 0 0]
  [0 0 0]
  [0 1 0]]

 [[0 1 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 0 1]
  [0 0 0]
  [0 1 0]]

 [[0 0 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 1]
  [0 0 0]
  [0 0 1]]

 [[0 0 1]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 1]
  [0 0 1]]

 [[1 0 0]
  [0 1 1]
  [0 0 0]]

 [[0 0 1]
  [1 0 0]
  [0 1 1]]

 [[0 0 1]
  [0 0 1]
  [1 0 0]]

 [[1 0 0]
  [0 0 1]
  [0 0 1]]

 [[0 0 1]
  [1 0 0]
  [0 0 1]]

 [[0 1 0]
  [0 0 1]
  [1 0 0]]

 [[0 0 1]
  [0 1 0]
  [0 0 1]]

 [[0 1 1]
  [0 0 1]
  [0 1 0]]

 [[0 1 0]
  [0 1 1]
  [0 0 1]]

 [[0 0 0]
  [0 1 0]
  [0 1 1]]

 [[0 0 1]
  [1 1 0]
  [0 0 0]]

 [[0 1 0]
  [0 0 1]
  [1 1 0]]

 [[0 1 1]
  [0 1 0]
  [0 0 1]]

 [[0 0 0]
  [0 1 1]
  [0 1 0]]

 [[0 1 1]
  [0 0 0]
  [0 1 1]]

 [[1 0 1]
  [0 1 1]
  [0 0 0]]

 [[0 1 1]
  [1 0 1]
  [0 1 1]]

 [[1 1 1]
  [0 1 1]
  [1 0 1]]

 [[1 0 0]
  [1 1 1]
  [0 1 1]]

 [[0 0 0]
  [1 0 0]
  [1 1 1]]

 [[0 1 0]
  [1 0 1]
  [0 0 0]]

 [[1 0 1

In [24]:
#training = [((1,1,0,0,0,1,0,0,0), 0)]
epochs = 500

game = make_game()

training = game.get_inputs()
outputs = game.play()
for i in range(1, epochs):
    if i%30 == 0:
        game = make_game()
    training = np.append(training, game.get_inputs(), axis=0)
    outputs = np.append(outputs, game.play(), axis=0)


images = training
target = outputs

In [25]:
print(__doc__)

# RBF

# The data that we are interested in is made of 8x8 images of digits, let's
# have a look at the first 3 images, stored in the `images` attribute of the
# dataset.  If we were working from image files, we could load them using
# pylab.imread.  Note that each image must have the same size. For these
# images, we know which digit they represent: it is given in the 'target' of
# the dataset.
#print(images)
#print("------")
#print(target)
#print("------------")
images_and_labels = list(zip(images, target))
#print(images_and_labels)
#print("---------------")
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(images)
data = images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples / 2], target[:n_samples / 2])

# Now predict the value of the digit on the second half:
expected = target[n_samples / 2:]
predicted = classifier.predict(data[n_samples / 2:])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

images_and_predictions = list(zip(images[n_samples / 4:], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

plt.show()

Automatically created module for IPython interactive environment


C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:36: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:37: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.02, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
             precision    recall  f1-score   support

          0       0.94      0.94      0.94     21441
          1       0.65      0.67      0.66      3559

avg / total       0.90      0.90      0.90     25000


Confusion matrix:
[[20172  1269]
 [ 1188  2371]]


C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:43: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [21]:
print(classifier.get_params())

{'max_iter': -1, 'C': 1.0, 'class_weight': None, 'degree': 3, 'kernel': 'rbf', 'tol': 0.001, 'random_state': None, 'decision_function_shape': None, 'shrinking': True, 'verbose': False, 'probability': False, 'coef0': 0.0, 'cache_size': 200, 'gamma': 0.001}


In [29]:
print(__doc__)

# SIGMOID

# The data that we are interested in is made of 8x8 images of digits, let's
# have a look at the first 3 images, stored in the `images` attribute of the
# dataset.  If we were working from image files, we could load them using
# pylab.imread.  Note that each image must have the same size. For these
# images, we know which digit they represent: it is given in the 'target' of
# the dataset.
#print(images)
#print("------")
#print(target)
#print("------------")
images_and_labels = list(zip(images, target))
#print(images_and_labels)
#print("---------------")
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(images)
data = images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(kernel="sigmoid", gamma = 0.001)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples / 2], target[:n_samples / 2])

# Now predict the value of the digit on the second half:
expected = target[n_samples / 2:]
predicted = classifier.predict(data[n_samples / 2:])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

print("--------")
print(predicted)

images_and_predictions = list(zip(images[n_samples / 4:], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

plt.show()

Automatically created module for IPython interactive environment


C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:36: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:37: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
             precision    recall  f1-score   support

          0       0.94      0.94      0.94     21441
          1       0.64      0.67      0.65      3559

avg / total       0.90      0.90      0.90     25000


Confusion matrix:
[[20089  1352]
 [ 1188  2371]]
--------
[0 0 0 ..., 1 1 0]


C:\Users\Luke\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [28]:
from sknn.mlp import Classifier, Convolution, Layer

#training = [((1,1,0,0,0,1,0,0,0), 0)]
epochs = 500

game = make_game()

training = game.get_inputs()
outputs = game.play()
for i in range(1, epochs):
    if i%30 == 0:
        game = make_game()
    training = np.append(training, game.get_inputs(), axis=0)
    outputs = np.append(outputs, game.play(), axis=0)


images = training
target = outputs



NameError: name 'precicted' is not defined

In [ ]:
#create a neural network
nn = Classifier(
    layers=[
        Convolution("Rectifier", channels=1, kernel_shape=(3,3)),
        Layer("Softmax")],
    learning_rate=0.02,
    n_iter=25)

game.draw(game.grid)
#debugging
nn.fit(images, outputs)



C:\Users\Luke\Anaconda3\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
C:\Users\Luke\Anaconda3\lib\site-packages\lasagne\layers\conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)
